# collect data from Twitter using Twitter API

## import packages and configure twitter api

In [1]:
# !pip install tweepy
import tweepy as tw
print(tw.__version__)

4.10.0


In [2]:
# !pip install pandas
import pandas as pd
import json
import time
import csv
import requests
print(requests.__version__)

2.28.1


In [3]:
# import keys from a .gitignore file

import keys

In [4]:
# assign the values accordingly

client = tw.Client(
    wait_on_rate_limit = True,
    consumer_key = keys.consumer_key,
    consumer_secret = keys.consumer_secret,
    access_token = keys.access_token,
    access_token_secret = keys.access_token_secret,
    bearer_token = keys.bearer_token,
)

In [5]:
# client = tw.Client(bearer_token)
client

## get the list of news media twitter username

In [7]:
# read a csv file with selected news media's twitter username 

news_media_basic_data = pd.read_csv (r'/Users/katherina/data_science_tweets_neutrality/news_media_twitter_username.csv')
news_media_basic_data

,news_media,twitter_username,us_or_foreign,general_or_political
0,ABC News,ABC,US,General
1,ABC News,ABCPolitics,US,Political
2,ABC News,ABCNewsLive,US,General
3,BBC News,BBCWorld,Foreign,General
4,BBC News,BBCBreaking,Foreign,General
5,BBC News,BBCNews,Foreign,General
6,BBC News,BBCNorthAmerica,Foreign,General
7,CBS News,CBSNews,US,General
8,CBS News,CBSPolitics,US,Political
9,CBS News,CBSNewsPoll,US,Political


In [8]:
news_media_twitter_username_list = news_media_basic_data['twitter_username'].tolist()
news_media_twitter_username_list

news_media_twitter_username_list = ["ABC","ABCPolitics","ABCNewsLive"]
news_media_twitter_username_list

['ABC', 'ABCPolitics', 'ABCNewsLive']

## Call Twitter API v2

### get user details

In [9]:
# call twitter API v2 to get user id and other user information for the user in the list

def get_users_details(usernames):
        users_details = client.get_users(usernames=usernames, user_fields=["created_at","location","protected","public_metrics","verified"])
        return users_details

twitter_user_details = get_users_details(news_media_twitter_username_list)
twitter_user_details

Response(data=[<User id=28785486 name=ABC News username=ABC>, <User id=16815644 name=ABC News Politics username=ABCPolitics>, <User id=384438102 name=ABC News Live username=ABCNewsLive>], includes={}, errors=[], meta={})

In [40]:
# create a list of records
twitter_user_details_lst = []

# iterate over each tweet and corresponding user details
for user in twitter_user_details.data:
    user_info = {
        'user_id': user.id,
        'user_name': user.name,
        'user_username': user.username,
        "user_created_at" : user.created_at,
        "user_location" : user.location,
        "user_protected" : user.protected,
        "user_followers_count" : user.public_metrics["followers_count"],
        "user_following_count" : user.public_metrics["following_count"],
        "user_tweet_count" : user.public_metrics["tweet_count"],
        "user_listed_count" : user.public_metrics["listed_count"],
        "user_verified" : user.verified
    }
    twitter_user_details_lst.append(user_info)user_tweets

# create dataframe from the extracted records
twitter_user_details_df = pd.DataFrame(twitter_user_details_lst)
# display the dataframe
twitter_user_details_df

,user_id,user_name,user_username,user_created_at,user_location,user_protected,user_followers_count,user_following_count,user_tweet_count,user_listed_count,user_verified
0,28785486,ABC News,ABC,2009-04-04 12:40:32+00:00,New York City / Worldwide,False,17625912,487,392310,66027,True
1,16815644,ABC News Politics,ABCPolitics,2008-10-16 22:43:16+00:00,"Washington, DC",False,1077940,474,192389,8485,True
2,384438102,ABC News Live,ABCNewsLive,2011-10-03 17:47:38+00:00,None,False,95749,223,36587,2235,True


In [6]:
twitter_user_details_df.to_csv (r'/Users/katherina/data_science_tweets_neutrality/twitter_user_details.csv', index = False, header=True)

NameError: name 'twitter_user_details_df' is not defined

In [ ]:
# for user in users.data:
#     print(user["id"])
#     print(user["created_at"])

In [33]:
# twitter_user_details.data[0]["location"]

In [6]:
twitter_user_details_data = pd.read_csv (r'/Users/katherina/data_science_tweets_neutrality/twitter_user_details.csv')
twitter_user_details_data

,user_id,user_name,user_username,user_created_at,user_location,user_protected,user_followers_count,user_following_count,user_tweet_count,user_listed_count,user_verified
0,28785486,ABC News,ABC,2009-04-04 12:40:32+00:00,New York City / Worldwide,False,17625912,487,392310,66027,True
1,16815644,ABC News Politics,ABCPolitics,2008-10-16 22:43:16+00:00,"Washington, DC",False,1077940,474,192389,8485,True
2,384438102,ABC News Live,ABCNewsLive,2011-10-03 17:47:38+00:00,NaN,False,95749,223,36587,2235,True


In [7]:
user_id_list = twitter_user_details_data["user_id"].tolist()
user_id_list

[28785486, 16815644, 384438102]

In [16]:
# Create funtions to get User's Tweets and save the results

# a function for gettting a user's tweet, input user_id, start_time, and end_time
def get_users_tweets(user_id,start_time,end_time):
    users_tweets = client.get_users_tweets(id=user_id,
                                           end_time=end_time, 
                                           exclude=None, 
                                           expansions="referenced_tweets.id", 
                                           max_results=5, 
                                           pagination_token=next_token,
                                           start_time=start_time, 
                                           tweet_fields=["author_id","context_annotations","created_at","entities","in_reply_to_user_id","lang","public_metrics","referenced_tweets"])
    return users_tweets

# a function to append get_users_tweets response to a lists
def user_tweets_response_to_lst(users_tweets,users_tweets_lst):

    for tweets in users_tweets.data:
        users_tweets_details = {
            'tweet_id': tweets.id,
            'tweet_text': tweets.text,
            'tweet_author_id': tweets.author_id,
            "tweet_context_annotations" : tweets.context_annotations,
            "tweet_created_at" : tweets.created_at,
            "tweet_entities" : tweets.entities,
            "tweet_in_reply_to_user_id" : tweets.in_reply_to_user_id,
            "tweet_lang" : tweets.lang,
            "tweet_retweet_count" : tweets.public_metrics["retweet_count"],
            "tweet_reply_count" : tweets.public_metrics["reply_count"],
            "tweet_like_count" : tweets.public_metrics["like_count"],
            "tweet_quote_count" : tweets.public_metrics["quote_count"],
            "tweet_referenced_tweets" : tweets.referenced_tweets
        }
        users_tweets_lst.append(users_tweets_details)
    return users_tweets_lst
    

In [17]:
# use get_users_tweets and user_tweets_response_to_lst to extract and save users' tweet in 2022 July

start_time = "2022-07-01T00:00:00Z"
end_time = "2022-08-01T00:00:00Z"

users_tweets_2022_july_lst = []

for user_id in user_id_list:
    finished = False
    next_token = None
    count = 0
    while finished is not True:
        users_tweets = get_users_tweets(user_id,start_time,end_time)
        users_tweets_2022_july_lst = user_tweets_response_to_lst(users_tweets,users_tweets_2022_july_lst)
        count = count+1

        if users_tweets.meta["next_token"] is not None and count < 2:
            next_token = users_tweets.meta["next_token"]
        else:
            finished = True
    

twitter_users_tweets_2022_july_df = pd.DataFrame(users_tweets_2022_july_lst)
twitter_users_tweets_2022_july_df

,tweet_id,tweet_text,tweet_author_id,tweet_context_annotations,tweet_created_at,tweet_entities,tweet_in_reply_to_user_id,tweet_lang,tweet_retweet_count,tweet_reply_count,tweet_like_count,tweet_quote_count,tweet_referenced_tweets
0,1553892491962843136,Smokers and vapers are more likely to have a s...,28785486,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",2022-07-31 23:56:54+00:00,"{'urls': [{'start': 111, 'end': 134, 'url': 'h...",NaN,en,59,43,144,16,None
1,1553886764930138122,Gun manufacturers have made more than $1 billi...,28785486,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",2022-07-31 23:34:09+00:00,"{'urls': [{'start': 214, 'end': 237, 'url': 'h...",NaN,en,83,389,188,65,None
2,1553880276702560256,Misinformation and stigma may be holding back ...,28785486,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",2022-07-31 23:08:22+00:00,"{'urls': [{'start': 69, 'end': 92, 'url': 'htt...",NaN,en,25,40,47,3,None
3,1553872564820361222,Rising concern over the impact of a potential ...,28785486,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",2022-07-31 22:37:43+00:00,"{'annotations': [{'start': 90, 'end': 96, 'pro...",NaN,en,24,28,59,7,None
4,1553865859977908225,"At least 28 people have been confirmed dead, b...",28785486,"[{'domain': {'id': '10', 'name': 'Person', 'de...",2022-07-31 22:11:05+00:00,"{'annotations': [{'start': 98, 'end': 109, 'pr...",NaN,en,83,37,203,6,None
5,1553861323691376648,A fire burning out of control in a Northern Ca...,28785486,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",2022-07-31 21:53:03+00:00,"{'urls': [{'start': 201, 'end': 224, 'url': 'h...",NaN,en,47,8,80,0,None
6,1553856250613534720,Two cyclists were killed and three others inju...,28785486,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",2022-07-31 21:32:54+00:00,"{'urls': [{'start': 160, 'end': 183, 'url': 'h...",NaN,en,27,5,46,1,None
7,1553847893634809858,Dramatic video captures the moment a portion o...,28785486,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",2022-07-31 20:59:41+00:00,"{'urls': [{'start': 196, 'end': 219, 'url': 'h...",NaN,en,31,5,53,3,None
8,1553844156761620485,"NEW: George Takei on Nichelle Nichols: ""My hea...",28785486,"[{'domain': {'id': '10', 'name': 'Person', 'de...",2022-07-31 20:44:50+00:00,"{'urls': [{'start': 130, 'end': 153, 'url': 'h...",28785486.0,en,114,19,826,3,"[(type, id)]"
9,1553839967796854785,NEW: Former Pres. Barack Obama on Bill Russell...,28785486,"[{'domain': {'id': '10', 'name': 'Person', 'de...",2022-07-31 20:28:11+00:00,"{'urls': [{'start': 186, 'end': 209, 'url': 'h...",NaN,en,1303,135,6068,55,None


In [13]:
twitter_users_tweets_2022_july_df.to_csv (r'/Users/katherina/data_science_tweets_neutrality/twitter_users_tweets_2022_july.csv', index = False, header=True)

In [20]:
users_tweets.meta["next_token"]

'7140dibdnow9c7btw422nhyc3xe5bcr6z6kxe16oy154l'

In [13]:
# search_words = "Joe Biden" # This will be changed to Donald Trump when we retrieve tweets related to him.
# date_since = "2020-10-27"
# tweets = tw.Cursor(api.search_tweets, q=search_words, lang="en").items(10)
# tweets

In [14]:
# tweet_details = [[tweet.geo, tweet.text, tweet.user.screen_name, tweet.user.location] for tweet in tweets]
# tweet_df = pd.DataFrame(data=tweet_details, columns=["geo","text","user","location"])
# tweet_df.head()

In [ ]:
# df.to_csv('data.csv')